In [404]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import pyecharts
from pyecharts import options as opts
from datetime import datetime
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
import keras
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.models import Model
from keras import regularizers
from keras.callbacks import TensorBoard
import xgboost

In [2]:
from pyecharts.charts import Line, Page

def line_with_constant_benchmark(data, title, benchmark=None, ratio=None, axis_offset=0, zoom=True, range_=None) -> Line:
    marklines = []
    if benchmark and ratio:
        marklines += [
            opts.MarkLineItem(y=benchmark, name="Benchmark"),
            opts.MarkLineItem(y=benchmark * (1 + ratio), name="Upper"),
            opts.MarkLineItem(y=benchmark * (1 - ratio), name="Lower"),
        ]
    if benchmark and not ratio:
        marklines += [
            opts.MarkLineItem(y=benchmark, name="Benchmark"),
        ]
    if range_ is not None:
        marklines += [
            opts.MarkLineItem(x=range_[0], name="Start"),
            opts.MarkLineItem(x=range_[1], name="End"),
        ]
    line_ = (
        Line(
            init_opts=opts.InitOpts(
                animation_opts=opts.AnimationOpts(
                    animation=False
                )
            ))
        .add_xaxis(range(len(data)))
        .add_yaxis(
            title,
            data[title].values,
            label_opts=opts.LabelOpts(is_show=False)
        ).set_global_opts(
            datazoom_opts=opts.DataZoomOpts() if zoom else None,
            yaxis_opts=opts.AxisOpts(
                min_=min(data[title].values) - axis_offset,
                max_=max(data[title].values) + axis_offset
            ),
        ).set_series_opts(
            label_opts=opts.LabelOpts(is_show=False),
            markline_opts=opts.MarkLineOpts(
                data=marklines
            ),
        )
    )
    return line_

def line_with_variated_benchmark(data, title, benchmark_title, axis_offset=0, zoom=True, range_=None) -> Line:
    marklines = []
    if range_ is not None:
        marklines += [
            opts.MarkLineItem(x=range_[0], name="Start"),
            opts.MarkLineItem(x=range_[1], name="End"),
        ]
    line_ = (
        Line(
            init_opts=opts.InitOpts(
                animation_opts=opts.AnimationOpts(
                    animation=False
                )
            ))
        .add_xaxis(range(len(data)))
        .add_yaxis(
            title,
            data[title].values,
            label_opts=opts.LabelOpts(is_show=False)
        )
        .add_yaxis(
            benchmark_title,
            data[benchmark_title].values,
            label_opts=opts.LabelOpts(is_show=False)
        ).set_global_opts(
            datazoom_opts=opts.DataZoomOpts() if zoom else None,
            yaxis_opts=opts.AxisOpts(
                min_=min(data[title].values) - axis_offset,
                max_=max(data[title].values) + axis_offset
            ),
        ).set_series_opts(
            markline_opts=opts.MarkLineOpts(
                data=marklines
            )
        )
    )
    return line_
    
    
def draw_split_data(split_data_item, title, zoom=True, range_=None, dir_=None):
    # split_data_item['入口水分'] = split_data_item['入口水分'].shift(-100, fill_value=0)
    page = Page()
    page.add(line_with_constant_benchmark(
        split_data_item, 
        "烘前叶丝流量",
        # np.mean(split_data_item['烘前叶丝流量设定值']), 
        axis_offset=5,
        zoom=zoom
    ))
    page.add(line_with_constant_benchmark(
        split_data_item, 
        "出口水分", 
        # np.mean(split_data_item['出口水分设定值']), 
        ratio=BIAS_ERROR_RATIO,
        zoom=zoom,
        range_=range_
    ))
    page.add(line_with_constant_benchmark(
        split_data_item, 
        "入口水分",
        zoom=zoom
    ))
    page.add(line_with_variated_benchmark(
        split_data_item, 
        "热风速度实际值", 
        '热风速度设定值',
        0.01,
        zoom=zoom,
        range_=range_
    ))
    page.add(line_with_variated_benchmark(
        split_data_item, 
        "筒壁1区温度实际值", 
        '筒壁1区温度设定值',
        0.5,
        zoom=zoom,
        range_=range_
    ))
    page.add(line_with_variated_benchmark(
        split_data_item, 
        "筒壁2区温度实际值",
        "筒壁2区温度设定值",
        0.5,
        zoom=zoom,
        range_=range_
    ))
    if dir_:
        make_dir('./plot/' + dir_ )
        page.render('./plot/'+ dir_ + '/'+ title + '.html')
    else:
        page.render('./plot/' + title + '.html')

def make_dir(path):
    if not os.path.exists(path):
        os.makedirs(path) 


## Read original Data

In [3]:
original = pd.read_csv('./data.csv', encoding='gbk')
original.columns.values

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1,2,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


array(['ID', '批次', '牌号', '时间', '生产班次', '生产班别', '设备状态', '烘前叶丝流量设定值',
       '烘前叶丝流量', '烘前叶丝流量累积量', 'SIROX蒸汽流量', '热风温度', '筒壁1区温度设定值',
       '筒壁1区温度实际值', '筒壁2区温度设定值', '筒壁2区温度实际值', '脱水量', '排潮风门开度', '罩压力',
       '热风速度设定值', '热风速度实际值', '出口温度', 'SIROX水分增加', '入口水分', '出口水分设定值',
       '出口水分', '冷凝水温度1区', '冷凝水温度2区', '滚筒转速', '蒸汽压力', '区域1预热阶段滚筒温度额定值',
       '区域2预热阶段滚筒温度额定值', '工作点脱水', '区域1滚筒温度标准工作点', '区域2滚筒温度标准工作点',
       '区域1筒壁蒸汽压力', '区域2筒壁蒸汽压力', '罩压力设定值'], dtype=object)

In [4]:
columns = [
 '时间', '牌号', '设备状态', \
 '入口水分', '出口水分', '出口水分设定值', \
 '热风速度设定值',  '热风速度实际值', \
 '烘前叶丝流量设定值', '烘前叶丝流量', \
 '筒壁1区温度设定值', '筒壁1区温度实际值',   \
 '筒壁2区温度实际值', '筒壁2区温度设定值'
]

data = original[columns]
data['出口水分差值'] = data['出口水分'] - data['出口水分设定值']

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


# 对总的数据进行预处理

In [5]:
# drop nan
data = data.dropna()

# 牌号莫名奇妙的存储时间，drop这些行
index = data[[isinstance(item, str) and item.startswith('2019') for item in data['牌号']]].index
data = data.drop(index, axis=0)

# 烘前叶丝流量 == 0，表示设备没有运行
index = data[data['烘前叶丝流量'] == 0].index
data = data.drop(index, axis=0)

# Formate Time
# def format_time(time_str):
#     try:
#         return datetime.strptime(time_str, '%Y-%m-%d %H:%M:%S')
#     except ValueError:
#         return datetime.strptime(time_str, '%Y-%m-%d')
# data['时间'] = data['时间'].map(lambda x: format_time(x))

# 按照时间进行排序
data = data.sort_values(by=['时间'], ascending=True)


In [6]:
# 查看设备状态的种类
for index, status in enumerate(data['设备状态'].unique()):
    print(status, ' ', data[data['设备状态'] == status].shape[0])

# 查看牌号的种类
for index, number in enumerate(data['牌号'].unique()):
    print(number, ' ', data[data['牌号'] == number].shape[0])

准备   11929
启动   170064
生产   1151154
收尾   2933
2048   2
-128   417
1   15
TG####A   448352
HSX###   153147
Txy###   146856
TH####A   341659
KPH###   9179
HsxY##   214655
DQMr##   18552
ThQD##A   4114


# 取得Sample Data + 预处理

In [7]:
data = data[data['设备状态'] == '生产']
sample_list = []
sample_list_name = []
FLOW_BIAS_DROP_RATE = 0.003

for index, number in enumerate(data['牌号'].unique()):
    sample_ = data[data['牌号'] == number]
    flow_mean = sample_['烘前叶丝流量设定值'].mean()
    flow_mask = np.abs(sample_['烘前叶丝流量'] - flow_mean) < FLOW_BIAS_DROP_RATE * flow_mean
    sample_ = sample_[flow_mask]
    sample_ = sample_.reset_index(drop=True) 
    print(number, sample_.shape)
    if sample_.shape[0] > 1000:
        sample_list.append(sample_)
        sample_list_name.append(number)
    

TG####A (320338, 15)
HSX### (133907, 15)
Txy### (101676, 15)
TH####A (251547, 15)
KPH### (4, 15)
HsxY## (187457, 15)
DQMr## (16128, 15)
ThQD##A (1966, 15)


In [8]:
sample_list[1][['入口水分', '出口水分', '热风速度实际值', '烘前叶丝流量', '筒壁1区温度实际值', '筒壁2区温度实际值']].describe()

,入口水分,出口水分,热风速度实际值,烘前叶丝流量,筒壁1区温度实际值,筒壁2区温度实际值
count,133907.000000,133907.000000,133907.000000,133907.000000,133907.000000,133907.000000
mean,21.724467,13.803235,0.388080,4799.990748,136.546417,136.550413
std,0.349600,0.078300,0.033596,4.100291,2.349876,2.350484
min,20.172890,13.290000,0.296220,4785.619000,130.442600,130.665400
25%,21.468455,13.750000,0.360193,4797.401000,134.807400,134.812800
50%,21.625880,13.800000,0.394065,4799.989000,135.746300,135.750200
75%,22.016550,13.850000,0.409710,4802.579000,138.877900,138.884750
max,22.703500,14.160000,0.467372,4814.381000,143.145800,143.224000


In [9]:
sample_list[1].tail(20)

,时间,牌号,设备状态,入口水分,出口水分,出口水分设定值,热风速度设定值,热风速度实际值,烘前叶丝流量设定值,烘前叶丝流量,筒壁1区温度设定值,筒壁1区温度实际值,筒壁2区温度实际值,筒壁2区温度设定值,出口水分差值
133887,2019-9-5 3:36:52,HSX###,生产,21.33688,13.69,13.8,0.44,0.438350,4800,4802.782,134.5552,134.0763,134.0882,134.5557,-0.11
133888,2019-9-5 3:36:54,HSX###,生产,21.35440,13.72,13.8,0.44,0.437822,4800,4805.456,134.5843,134.0629,134.0845,134.6058,-0.08
133889,2019-9-5 3:36:56,HSX###,生产,21.35762,13.70,13.8,0.44,0.438635,4800,4806.367,134.6292,134.0548,134.0820,134.6673,-0.10
133890,2019-9-5 3:36:58,HSX###,生产,21.35528,13.66,13.8,0.44,0.439288,4800,4807.084,134.6719,134.0463,134.0841,134.7219,-0.14
133891,2019-9-5 3:37:00,HSX###,生产,21.35578,13.64,13.8,0.44,0.439629,4800,4809.135,134.6709,134.0417,134.0886,134.7208,-0.16
133892,2019-9-5 3:37:02,HSX###,生产,21.35930,13.65,13.8,0.44,0.439927,4800,4808.049,134.6359,134.0424,134.0965,134.6937,-0.15
133893,2019-9-5 3:37:04,HSX###,生产,21.37075,13.65,13.8,0.44,0.440603,4800,4809.277,134.6043,134.0411,134.1073,134.6688,-0.15
133894,2019-9-5 3:37:06,HSX###,生产,21.38248,13.60,13.8,0.44,0.440897,4800,4806.968,134.5871,134.0405,134.1179,134.6604,-0.20
133895,2019-9-5 3:37:08,HSX###,生产,21.37599,13.64,13.8,0.44,0.440865,4800,4806.898,134.5182,134.0411,134.1295,134.6032,-0.16
133896,2019-9-5 3:37:10,HSX###,生产,21.37023,13.61,13.8,0.44,0.439706,4800,4804.588,134.4811,134.0381,134.1433,134.5818,-0.19


# 按时间进行分割

In [10]:
def format_time(time_str):
    try:
        return datetime.strptime(time_str, '%Y-%m-%d %H:%M:%S')
    except ValueError:
        return datetime.strptime(time_str, '%Y-%m-%d')

for index, sample in enumerate(sample_list):
    sample_list[index]['时间'] = sample['时间'].map(lambda x: format_time(x))


In [11]:
feature_column = ['出口水分差值', '热风速度实际值', '筒壁1区温度实际值', '筒壁2区温度实际值']
label_column = ['热风速度设定值', '筒壁1区温度设定值', '筒壁2区温度设定值']

# 5   3   50   6
TIME_LAG_STEP = 5
TIME_LAG_1_END = 3
TIME_LAG_1_START = 25
STABLE_WINDOWS_SIZE = 5

TIME_LAG_2 = 100
BIAS_ERROR_RATIO = 0.002
LABLE_WINDOWS_SIZE = 1

In [12]:
# 如果两个点相差 SPLIT_INTERVAL 秒，进行分割
SPLIT_INTERVAL = 300
split_data_per_number = []

for sample in sample_list:
    sample_time_diff = (sample['时间'][1:] - sample['时间'].shift(1)[1:]).map(lambda x: x.seconds)

    split_point = sample_time_diff[sample_time_diff > SPLIT_INTERVAL].index
    split_point = split_point.insert(0, 0)
    split_point = split_point.insert(len(split_point), len(sample))
    split_data = []
    for i in range(1, len(split_point)):
        if split_point[i] - split_point[i - 1] > TIME_LAG_2:
            split_data.append(sample[split_point[i - 1]: split_point[i]])
    split_data_per_number.append(split_data)


# 画图

In [13]:
make_dir("./plot")
draw_split_data(split_data_per_number[0][54], '54')

# 抽取训练数据
选取出口水分在STABLE_WINDOWS_SIZE内都是稳定的一段时间，然后向前找到 T-TIME_LAG_1_START 到 T-TIME_LAG_1_END 时间内的所有特征，来预测 T-TIME_LAG_1_END 时刻需要调整的参数值

Label即 T-TIME_LAG_1_END 到 T 时刻的各种参数的差值

In [14]:
def calc_feature(item_,
                 stable_start: int,
                 time_lag_2=TIME_LAG_2,
                 time_lag_1_start=TIME_LAG_1_START,
                 time_lag_1_end=TIME_LAG_1_END) -> []:
    input_humidity = item_['入口水分'].iloc[stable_start - time_lag_2 - time_lag_1_start: stable_start - time_lag_2 - time_lag_1_end]
    feature_slice = item_[feature_column].iloc[stable_start - time_lag_1_start: stable_start - time_lag_1_end]

    return np.concatenate([
        [
            calc_integral(input_humidity.values),
            input_humidity.std(),
            input_humidity.skew(),
            input_humidity.kurtosis(),
        ],
        calc_integral(feature_slice.values),
        feature_slice.mean().values,
        feature_slice.std().values,
        feature_slice.skew().values,
        feature_slice.kurtosis().values,
        [time_lag_1_start - time_lag_1_end]
    ])
    

def calc_integral(data):
    if len(data) <= 1:
        return 0
    sum_ = sum(data)
    return sum_ - (data[0] + data[len(data) - 1]) / 2

def calc_lable(item_, end: int) -> []:
    real_start = end
    # real_end = end + STABLE_WINDOWS_SIZE
    
    current_start = end - TIME_LAG_1_END
    # current_end = end - TIME_LAG_1_END + LABLE_WINDOWS_SIZE
    
    real_ = np.mean(item_[label_column].iloc[current_start: real_start].values, axis=0)
    current_ = item_[label_column].iloc[current_start]
    return real_ - current_

In [15]:
def generate_data(split_data, split_data_index):
    setting = np.mean(split_data[0]['出口水分设定值'])
    sample_train_dataset = []
    sample_train_label = []
    sample_data_windows = []
    
    for index, item in enumerate(split_data):
        length = len(item)
        humidity = item['出口水分']
        wind_speed = item['热风速度设定值']

        # 保证出口水分在一定时间的恒定值内
        for stable_start in range(TIME_LAG_2 + TIME_LAG_1_START, length - STABLE_WINDOWS_SIZE):
            stable_end = stable_start + STABLE_WINDOWS_SIZE
            if np.all(np.abs(humidity[stable_start: stable_end] - setting) < setting * BIAS_ERROR_RATIO):
                
                for time_lag_1_start_step in range(TIME_LAG_1_START, TIME_LAG_1_END + 15, -TIME_LAG_STEP):
                    sample_data_windows.append([split_data_index, index, stable_start])
                    sample_train_dataset.append(calc_feature(item, stable_start, TIME_LAG_2, time_lag_1_start_step, TIME_LAG_1_END))
                    sample_train_label.append(calc_lable(item, stable_start))

        # 保证风速调整后，出口水分是正常的
        for adjust_start in range(TIME_LAG_2 + TIME_LAG_1_START, length - TIME_LAG_1_END):
            if np.abs(wind_speed.iloc[adjust_start - 1] - wind_speed.iloc[adjust_start]) > 0 \
                    and np.abs(item['出口水分'] - setting).iloc[TIME_LAG_1_END + adjust_start] < setting * BIAS_ERROR_RATIO:

                for time_lag_1_start_step in range(TIME_LAG_1_START, TIME_LAG_1_END + 15, -TIME_LAG_STEP):
                    sample_data_windows.append([split_data_index, index, adjust_start + TIME_LAG_1_END])
                    sample_train_dataset.append(calc_feature(item, adjust_start + TIME_LAG_1_END, TIME_LAG_2, time_lag_1_start_step, TIME_LAG_1_END))
                    sample_train_label.append(calc_lable(item, adjust_start + TIME_LAG_1_END))
    
    sample_train_dataset = np.array(sample_train_dataset)
    sample_train_label = np.array(sample_train_label)
    sample_data_windows = np.array(sample_data_windows)
    
    return sample_train_dataset, sample_train_label, sample_data_windows

In [16]:
def inconsistent_symbol(label_):
        return np.array(label_[:, 0] * label_[:, 1] < 0) \
               | np.array(label_[:, 0] * label_[:, 2] < 0) \
               | np.array(label_[:, 1] * label_[:, 2] < 0)
    
def remove_inconsistent(sample_train_dataset, sample_train_label, sample_data_windows):
    # 去除风速非常小的数据
    sample_train_label[np.logical_and(sample_train_label < 1e-8, sample_train_label > -1e-8)] = 0

    # 去除风速，温度符号不同的数值
    inconsistent_mask = inconsistent_symbol(sample_train_label)
    # print('remove inconsistent: ', sum(inconsistent_mask))

    sample_train_label = sample_train_label[~inconsistent_mask]
    sample_train_dataset = sample_train_dataset[~inconsistent_mask]
    sample_data_windows = sample_data_windows[~inconsistent_mask]
    return sample_train_dataset, sample_train_label, sample_data_windows

# sample_train_dataset, sample_train_label, sample_data_windows = remove_inconsistent(sample_train_dataset, sample_train_label, sample_data_windows)

In [17]:
# train_dataset_list = np.load('./npy/train_dataset_list.npy', allow_pickle=True)
# train_label_list = np.load('./npy/train_label_list.npy', allow_pickle=True)
# data_windows_list = np.load('./npy/data_windows_list.npy', allow_pickle=True)

# np.save('./npy/train_dataset_list_1.npy', arr=train_dataset_list)
# np.save('./npy/train_label_list_1.npy', arr=train_label_list)
# np.save('./npy/data_windows_list_1.npy', arr=data_windows_list)

train_dataset_list = []
train_label_list = []
data_windows_list = []


In [18]:
for split_data_index, split_data in enumerate(split_data_per_number):
    start = datetime.now()
    
    sample_train_dataset, sample_train_label, sample_data_windows = generate_data(split_data, split_data_index)
    sample_train_dataset, sample_train_label, sample_data_windows = remove_inconsistent(sample_train_dataset, sample_train_label, sample_data_windows)

    train_dataset_list.append(sample_train_dataset)
    train_label_list.append(sample_train_label)
    data_windows_list.append(sample_data_windows)
    
    print(split_data_index, ':', len(sample_train_dataset))
    print('time:', datetime.now() - start)


0 : 55210
time: 0:06:41.112226
1 : 17878
time: 0:02:29.262662
2 : 22904
time: 0:02:33.753245
3 : 44958
time: 0:05:17.022580
4 : 29938
time: 0:03:50.280715
5 : 2682
time: 0:00:24.448737
6 : 390
time: 0:00:02.535079


In [19]:
sample_train_dataset = train_dataset_list[0]
sample_train_label = train_label_list[0]
sample_data_windows = data_windows_list[0]

for i in range(1, len(train_dataset_list)):
    sample_train_dataset = np.concatenate([sample_train_dataset, train_dataset_list[i]], axis=0)
    sample_train_label = np.concatenate([sample_train_label, train_label_list[i]], axis=0)
    sample_data_windows = np.concatenate([sample_data_windows, data_windows_list[i]], axis=0)


In [261]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test, index_train, index_test = train_test_split(sample_train_dataset, sample_train_label, sample_data_windows, test_size=0.1, random_state=42)

print('Train set: ', len(X_train))
print('Val set: ', len(X_test))

Train set:  156564
Val set:  17396


In [21]:
pd.DataFrame(sample_train_dataset).corr()
# ['入口水分', '出口水分差值', '热风速度实际值', '筒壁1区温度实际值', '筒壁2区温度实际值']
# ['积分', '方差', ‘Skew’, 'Kurtosis']
# ['积分'，'均值', '方差', ‘Skew’, 'Kurtosis']

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,1.000000,0.159831,-0.003098,-0.000878,0.004424,0.811157,0.908719,0.992868,-0.009406,0.029237,...,0.189764,0.003312,-0.005476,-0.001772,-0.008516,-0.010986,0.010389,0.003886,0.008201,0.942825
1,0.159831,1.000000,-0.010505,-0.260680,0.013283,0.101474,0.120689,0.154976,0.011441,0.008763,...,0.057979,0.010396,0.008218,0.003745,0.017744,-0.004922,-0.002968,-0.000073,0.007951,0.114831
2,-0.003098,-0.010505,1.000000,-0.033388,-0.003497,0.001313,-0.003987,-0.001360,-0.003668,0.007011,...,0.008209,0.002916,-0.002950,-0.017810,0.012380,0.003807,0.003176,-0.007288,-0.003634,-0.002788
3,-0.000878,-0.260680,-0.033388,1.000000,-0.001255,-0.002314,0.001171,-0.001409,-0.000882,-0.001486,...,0.004150,0.001828,-0.017617,0.003647,-0.003947,-0.005706,-0.008313,0.003613,-0.004103,-0.001749
4,0.004424,0.013283,-0.003497,-0.001255,1.000000,-0.025712,0.034418,0.019218,0.990697,-0.069869,...,0.036564,-0.155249,0.008489,0.006747,-0.014529,-0.016027,-0.003773,0.006685,-0.002014,0.014509
5,0.811157,0.101474,0.001313,-0.002314,-0.025712,1.000000,0.830798,0.810386,-0.037907,0.532228,...,0.118676,-0.004251,-0.005705,0.004290,-0.000721,-0.008854,0.012320,0.004368,0.004935,0.843470
6,0.908719,0.120689,-0.003987,0.001171,0.034418,0.830798,1.000000,0.927847,0.020371,0.026006,...,0.158279,0.000273,-0.004161,-0.006432,-0.008937,-0.007420,0.016994,0.004574,0.004322,0.968184
7,0.992868,0.154976,-0.001360,-0.001409,0.019218,0.810386,0.927847,1.000000,0.005270,0.010473,...,0.185498,0.002575,-0.004522,-0.010987,-0.015616,-0.009654,0.011630,0.003876,0.007185,0.953969
8,-0.009406,0.011441,-0.003668,-0.000882,0.990697,-0.037907,0.020371,0.005270,1.000000,-0.071004,...,0.034179,-0.148320,0.008488,0.001643,-0.020167,-0.016133,-0.003811,0.006808,-0.002194,-0.000040
9,0.029237,0.008763,0.007011,-0.001486,-0.069869,0.532228,0.026006,0.010473,-0.071004,1.000000,...,-0.038083,-0.007371,-0.003007,0.005501,-0.000460,-0.007804,-0.001199,0.000348,-0.000477,-0.000129


# 定义模型

In [293]:
from sklearn.linear_model import LinearRegression

clf = LinearRegression()
clf.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [355]:
pred = clf.predict(X_test)
pred[np.logical_and(pred < 1e-4, pred > -1e-4)] = 0
print('mae: ', round(mean_absolute_error(y_test, pred), 5))
print('r2: ', round(r2_score(y_test, pred), 5))

mae:  0.01089
r2:  0.16322


In [392]:
kernel_ = np.array(clf.coef_).T
bias_ = np.array(clf.intercept_)
def kernel_initializer(shape, dtype=None):
    return kernel_

def bias_initializer(shape, dtype=None):
    return bias_

def loss(y_true, y_pred):
    
    diff = K.abs((y_true - y_pred) * y_true)
    
    symbol_diff = -y_pred * y_true / K.clip(K.abs(y_pred * y_true), K.epsilon(), None)
#     wind_coef = K.clip(K.maximum(-y_pred[:, 0] * y_true[:, 0], 0), K.epsilon(), None) * 1e7
    wind_coef = K.relu(symbol_diff[:, 0])
#     temperature_coef = K.clip(K.maximum(-y_pred[:, 1:] * y_true[:, 1:], 0), K.epsilon(), None) * 1e7
    temperature_coef = K.relu(symbol_diff[:, 1:], 0) * 1e7
    
#     return K.mean(temperature_coef, axis=-1)

    return K.mean(diff[:, 0], axis=-1) + K.mean(diff[:, 1:], axis=-1)


In [300]:
def compute_loss(y_true, y_pred):
    diff = np.abs((y_true - y_pred) * np.clip(np.abs(y_true) * 1e3, 1e-7, None))
    
    symbol_diff = -y_pred * y_true / np.clip(np.abs(y_pred * y_true), 1e-7, None)
    wind_coef = np.clip(np.maximum(symbol_diff[:, 0], 0), 1e-7, None)
    temperature_coef = np.clip(np.maximum(-symbol_diff[:, 1:], 0), 1e-7, None)
    print(wind_coef, temperature_coef)
    return np.mean(diff[:, 0] + wind_coef, axis=-1) + np.mean(diff[:, 1:] + temperature_coef, axis=-1)

y_true = np.array([[0, 0.0046, 0.0044]])
y_pred = np.array([[0.000, 0.1678, 0.0166 ]])
compute_loss(y_true, y_pred)

[0.0000001] [[1. 1.]]


array([1.4022001])

In [429]:
model = Sequential()
model.add(Dense(input_dim=25, units=3, kernel_initializer=kernel_initializer, bias_initializer=bias_initializer))


model.compile(optimizer=keras.optimizers.Adam(), loss=loss, metrics=['mae'])
# model.summary()

now_ = str(datetime.now())
if not os.path.exists('./ckpt/' + now_):
        os.makedirs('./ckpt/' + now_) 

model.fit(X_train,
        y_train, 
        batch_size=2048 * 2, 
        epochs=3000,
        verbose=1,
        validation_data=(X_test, y_test),
        callbacks = [
            keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=30, min_lr=1e-12),
            keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=60, verbose=0, mode='min'),
            TensorBoard(log_dir='./tensorboard/' + "{0:%Y-%m-%d %H:%M:%S/}".format(datetime.now())),
            keras.callbacks.ModelCheckpoint(filepath='./ckpt/' + now_ + '/model.{epoch:03d}-{loss:.4f}.h5', monitor='val_loss', save_best_only=True, verbose=0)
        ])

Train on 156564 samples, validate on 17396 samples
Epoch 1/3000
156564/156564 [==============================] - 2s 13us/step - loss: 26811.6991 - mae: 1.2504 - val_loss: 12952.4476 - val_mae: 0.9034
Epoch 2/3000
156564/156564 [==============================] - 0s 2us/step - loss: 17033.0899 - mae: 1.0666 - val_loss: 6902.5410 - val_mae: 0.3404
Epoch 3/3000
156564/156564 [==============================] - 0s 2us/step - loss: 16741.1314 - mae: 0.6810 - val_loss: 21290.4744 - val_mae: 0.7893
Epoch 4/3000
156564/156564 [==============================] - 0s 2us/step - loss: 16520.8542 - mae: 0.7366 - val_loss: 26743.4922 - val_mae: 1.3381
Epoch 5/3000
156564/156564 [==============================] - 0s 2us/step - loss: 16969.8530 - mae: 0.7705 - val_loss: 20199.8266 - val_mae: 0.8898
Epoch 6/3000
156564/156564 [==============================] - 0s 2us/step - loss: 17230.3425 - mae: 0.7337 - val_loss: 4075.0019 - val_mae: 0.4493
Epoch 7/3000
156564/156564 [==============================] - 

156564/156564 [==============================] - 0s 2us/step - loss: 5734.1839 - mae: 0.4418 - val_loss: 5859.9363 - val_mae: 0.4324
Epoch 57/3000
156564/156564 [==============================] - 0s 2us/step - loss: 5680.6722 - mae: 0.4254 - val_loss: 6841.3436 - val_mae: 0.3422
Epoch 58/3000
156564/156564 [==============================] - 0s 2us/step - loss: 5717.9451 - mae: 0.3935 - val_loss: 4924.2555 - val_mae: 0.5230
Epoch 59/3000
156564/156564 [==============================] - 0s 2us/step - loss: 5700.8494 - mae: 0.4119 - val_loss: 5432.9899 - val_mae: 0.3991
Epoch 60/3000
156564/156564 [==============================] - 0s 2us/step - loss: 5721.4452 - mae: 0.3262 - val_loss: 6964.5027 - val_mae: 0.3921
Epoch 61/3000
156564/156564 [==============================] - 0s 2us/step - loss: 6391.4367 - mae: 0.3777 - val_loss: 20178.6359 - val_mae: 1.0127
Epoch 62/3000
156564/156564 [==============================] - 0s 2us/step - loss: 8174.3823 - mae: 0.5707 - val_loss: 8128.3298 - 

156564/156564 [==============================] - 0s 2us/step - loss: 4034.3956 - mae: 0.2956 - val_loss: 3996.7273 - val_mae: 0.3411
Epoch 112/3000
156564/156564 [==============================] - 0s 2us/step - loss: 4063.1080 - mae: 0.2408 - val_loss: 2449.2525 - val_mae: 0.1193
Epoch 113/3000
156564/156564 [==============================] - 0s 2us/step - loss: 4066.8285 - mae: 0.2833 - val_loss: 2614.6059 - val_mae: 0.2669
Epoch 114/3000
156564/156564 [==============================] - 0s 2us/step - loss: 4048.3159 - mae: 0.2570 - val_loss: 2974.4430 - val_mae: 0.1025
Epoch 115/3000
156564/156564 [==============================] - 0s 2us/step - loss: 4045.2033 - mae: 0.2043 - val_loss: 2823.3629 - val_mae: 0.2392
Epoch 116/3000
156564/156564 [==============================] - 0s 2us/step - loss: 4087.5069 - mae: 0.3050 - val_loss: 1711.8490 - val_mae: 0.3235
Epoch 117/3000
156564/156564 [==============================] - 0s 1us/step - loss: 4991.5555 - mae: 0.4032 - val_loss: 3431.76

156564/156564 [==============================] - 0s 2us/step - loss: 1033.5721 - mae: 0.0632 - val_loss: 1310.7538 - val_mae: 0.0471
Epoch 167/3000
156564/156564 [==============================] - 0s 2us/step - loss: 1044.5760 - mae: 0.1135 - val_loss: 944.3081 - val_mae: 0.0436
Epoch 168/3000
156564/156564 [==============================] - 0s 2us/step - loss: 1054.2597 - mae: 0.0881 - val_loss: 1189.7594 - val_mae: 0.0453
Epoch 169/3000
156564/156564 [==============================] - 0s 2us/step - loss: 1071.5198 - mae: 0.0842 - val_loss: 820.7892 - val_mae: 0.0506
Epoch 170/3000
156564/156564 [==============================] - 0s 2us/step - loss: 1042.8385 - mae: 0.0850 - val_loss: 906.0415 - val_mae: 0.0470
Epoch 171/3000
156564/156564 [==============================] - 0s 2us/step - loss: 1056.1355 - mae: 0.0796 - val_loss: 815.9235 - val_mae: 0.0518
Epoch 172/3000
156564/156564 [==============================] - 0s 2us/step - loss: 953.8169 - mae: 0.0790 - val_loss: 830.6857 - v

Epoch 222/3000
156564/156564 [==============================] - 0s 2us/step - loss: 667.3217 - mae: 0.0458 - val_loss: 718.7783 - val_mae: 0.0446
Epoch 223/3000
156564/156564 [==============================] - 0s 2us/step - loss: 658.8208 - mae: 0.0390 - val_loss: 641.3364 - val_mae: 0.0485
Epoch 224/3000
156564/156564 [==============================] - 0s 2us/step - loss: 661.9762 - mae: 0.0416 - val_loss: 802.5280 - val_mae: 0.0394
Epoch 225/3000
156564/156564 [==============================] - 0s 2us/step - loss: 679.5186 - mae: 0.0455 - val_loss: 615.6355 - val_mae: 0.0293
Epoch 226/3000
156564/156564 [==============================] - 0s 2us/step - loss: 698.6291 - mae: 0.0300 - val_loss: 791.9331 - val_mae: 0.0332
Epoch 227/3000
156564/156564 [==============================] - 0s 2us/step - loss: 674.3962 - mae: 0.0498 - val_loss: 663.5823 - val_mae: 0.0456
Epoch 228/3000
156564/156564 [==============================] - 0s 2us/step - loss: 677.3979 - mae: 0.0450 - val_loss: 695.9

...

156564/156564 [==============================] - 0s 2us/step - loss: 493.8573 - mae: 0.0174 - val_loss: 536.8164 - val_mae: 0.0153
Epoch 335/3000
156564/156564 [==============================] - 0s 1us/step - loss: 499.3046 - mae: 0.0164 - val_loss: 506.7034 - val_mae: 0.0124
Epoch 336/3000
156564/156564 [==============================] - 0s 2us/step - loss: 505.2581 - mae: 0.0161 - val_loss: 502.0715 - val_mae: 0.0122
Epoch 337/3000
156564/156564 [==============================] - 0s 2us/step - loss: 493.6529 - mae: 0.0186 - val_loss: 514.8304 - val_mae: 0.0186
Epoch 338/3000
156564/156564 [==============================] - 0s 2us/step - loss: 497.1943 - mae: 0.0157 - val_loss: 509.0168 - val_mae: 0.0193
Epoch 339/3000
156564/156564 [==============================] - 0s 2us/step - loss: 493.2988 - mae: 0.0169 - val_loss: 504.2177 - val_mae: 0.0144
Epoch 340/3000
156564/156564 [==============================] - 0s 2us/step - loss: 495.6832 - mae: 0.0159 - val_loss: 512.1530 - val_mae: 

156564/156564 [==============================] - 0s 2us/step - loss: 495.2800 - mae: 0.0140 - val_loss: 516.5154 - val_mae: 0.0128
Epoch 391/3000
156564/156564 [==============================] - 0s 2us/step - loss: 494.6758 - mae: 0.0135 - val_loss: 536.3862 - val_mae: 0.0158
Epoch 392/3000
156564/156564 [==============================] - 0s 2us/step - loss: 493.6911 - mae: 0.0140 - val_loss: 504.2819 - val_mae: 0.0128
Epoch 393/3000
156564/156564 [==============================] - 0s 2us/step - loss: 495.7797 - mae: 0.0144 - val_loss: 507.8208 - val_mae: 0.0149
Epoch 394/3000
156564/156564 [==============================] - 0s 2us/step - loss: 492.4888 - mae: 0.0145 - val_loss: 503.9886 - val_mae: 0.0139
Epoch 395/3000
156564/156564 [==============================] - 0s 2us/step - loss: 493.1630 - mae: 0.0155 - val_loss: 505.8325 - val_mae: 0.0123
Epoch 396/3000
156564/156564 [==============================] - 0s 2us/step - loss: 491.3764 - mae: 0.0151 - val_loss: 503.0630 - val_mae: 

156564/156564 [==============================] - 0s 2us/step - loss: 490.2142 - mae: 0.0121 - val_loss: 502.5751 - val_mae: 0.0120
Epoch 447/3000
156564/156564 [==============================] - 0s 2us/step - loss: 490.0873 - mae: 0.0122 - val_loss: 502.4143 - val_mae: 0.0123
Epoch 448/3000
156564/156564 [==============================] - 0s 2us/step - loss: 490.3598 - mae: 0.0124 - val_loss: 502.3919 - val_mae: 0.0125
Epoch 449/3000
156564/156564 [==============================] - 0s 2us/step - loss: 489.7770 - mae: 0.0122 - val_loss: 501.7500 - val_mae: 0.0127
Epoch 450/3000
156564/156564 [==============================] - 0s 2us/step - loss: 490.0312 - mae: 0.0125 - val_loss: 501.8091 - val_mae: 0.0121
Epoch 451/3000
156564/156564 [==============================] - 0s 2us/step - loss: 490.1996 - mae: 0.0135 - val_loss: 502.7043 - val_mae: 0.0120
Epoch 452/3000
156564/156564 [==============================] - 0s 2us/step - loss: 490.8997 - mae: 0.0132 - val_loss: 502.0008 - val_mae: 

156564/156564 [==============================] - 0s 2us/step - loss: 489.8259 - mae: 0.0121 - val_loss: 501.6796 - val_mae: 0.0119
Epoch 503/3000
156564/156564 [==============================] - 0s 2us/step - loss: 490.6735 - mae: 0.0120 - val_loss: 502.0889 - val_mae: 0.0118
Epoch 504/3000
156564/156564 [==============================] - 0s 2us/step - loss: 490.0432 - mae: 0.0119 - val_loss: 502.1560 - val_mae: 0.0121
Epoch 505/3000
156564/156564 [==============================] - 0s 2us/step - loss: 490.2832 - mae: 0.0121 - val_loss: 502.5003 - val_mae: 0.0119
Epoch 506/3000
156564/156564 [==============================] - 0s 2us/step - loss: 489.9973 - mae: 0.0120 - val_loss: 501.9044 - val_mae: 0.0121
Epoch 507/3000
156564/156564 [==============================] - 0s 2us/step - loss: 489.8612 - mae: 0.0121 - val_loss: 502.5817 - val_mae: 0.0125
Epoch 508/3000
156564/156564 [==============================] - 0s 2us/step - loss: 489.8457 - mae: 0.0120 - val_loss: 501.6849 - val_mae: 

KeyboardInterrupt: 

In [430]:
pred = model.predict(X_test)
pred[np.logical_and(pred < 1e-4, pred > -1e-4)] = 0
print('mae: ', round(mean_absolute_error(y_test, pred), 5))
print('r2: ', round(r2_score(y_test, pred), 5))

mae:  0.0118
r2:  -5.99492


# 验证集阶段

In [431]:
np.set_printoptions(suppress=True)

def print_result(range_=100, dir_=None, save_result=False):
    str_ = []
    for i in range(range_):
        item = split_data_per_number[index_test[i][0]][index_test[i][1]]
        item = item[index_test[i][2] - TIME_LAG_2: index_test[i][2] + STABLE_WINDOWS_SIZE]
        
#         draw_split_data(item, 
#                         str(index_test[i][0]) + '-' + str(index_test[i][1]) + '-' + str(index_test[i][2]), 
#                         zoom=False, 
#                         range_=[TIME_LAG_2 - TIME_LAG_1_START, TIME_LAG_2 - TIME_LAG_1_END],
#                         dir_=dir_
#                        )
        if not save_result:
            if np.logical_or(np.array(pred[i, 1] * y_test[i, 1] < 0), np.array(pred[i, 2] * y_test[i, 2] < 0)):
                print('{}-{}-{}: \t {}, \t{} \t *'.format(index_test[i][0], index_test[i][1], index_test[i][2], np.round(pred[i], 4), np.round(y_test[i], 4)))
            else:
                print('{}-{}-{}: \t {}, \t{}'.format(index_test[i][0], index_test[i][1], index_test[i][2], np.round(pred[i], 4), np.round(y_test[i], 4)))
        else:
            str_.append(str(index_test[i][0]) + '-' + str(index_test[i][1]) + ': [' + ' '.join([str(x) for x in np.round(pred[i], 4)]) + '], [' + ' '.join([str(x) for x in np.round(y_test[i], 4)]) + ']')
    
    if save_result:
        fw = open('./plot/' + dir_ + '/result.txt', 'w')
        for line in str_:
            fw.write(line)
            fw.write("\n") 

print_result(dir_='val/' + str(datetime.now()))

0-215-533: 	 [-0.0005 -0.0008 -0.0011], 	[0.     0.0113 0.0106] 	 *
0-103-1458: 	 [ 0.0004 -0.0147 -0.0154], 	[ 0.     -0.0284 -0.0262]
0-91-1133: 	 [0.     0.005  0.0056], 	[ 0.     -0.0253 -0.0009] 	 *
0-37-265: 	 [-0.0003  0.0157  0.0164], 	[0.     0.0007 0.0035]
4-62-1527: 	 [-0.0003 -0.0067 -0.006 ], 	[0.0001 0.0011 0.0031] 	 *
0-149-360: 	 [0.0002 0.0068 0.0066], 	[0.     0.0179 0.027 ]
3-127-374: 	 [-0.0003  0.0109  0.0143], 	[ 0.     -0.0177 -0.0078] 	 *
0-209-919: 	 [-0.0002 -0.0079 -0.0061], 	[ 0.     -0.0424 -0.0175]
0-208-316: 	 [0.0003 0.0348 0.0344], 	[0.     0.0713 0.0736]
4-24-567: 	 [ 0.0002 -0.0079 -0.0065], 	[ 0.     -0.0293 -0.0175]
5-5-321: 	 [-0.0003  0.0032  0.003 ], 	[0.     0.0054 0.0203]
1-57-1738: 	 [-0.0008 -0.016  -0.0161], 	[ 0.     -0.0243 -0.0143]
0-43-746: 	 [-0.0006 -0.0074 -0.0094], 	[0.     0.0145 0.016 ] 	 *
0-241-1227: 	 [ 0.0005 -0.0069 -0.0078], 	[0.     0.0264 0.0257] 	 *
4-37-1555: 	 [-0.0004 -0.0015 -0.0057], 	[0.     0.0209 0.0033] 	 *
3-83-1

# 测试集合测试
滑动窗口不断滑动，然后取数据进行测试

In [321]:
def generate_real_test_data(item):
    length = len(item)
    final_X_test_ = []
    final_X_index_ = []
    
    for item_index in range(TIME_LAG_2 + TIME_LAG_1_START, length - STABLE_WINDOWS_SIZE, 1):
        final_X_test_.append(calc_feature(item, item_index))
        final_X_index_.append(item_index)
        
    return np.array(final_X_test_), np.array(final_X_index_)

final_test_data = split_data_per_number[0][5]
final_X_test, final_X_index = generate_real_test_data(final_test_data)
pred = model.predict(final_X_test)

dir_ = 'final/' + str(datetime.now())

# Plot
for index_, item_ in enumerate(final_X_index):
    draw_split_data(final_test_data[item_- TIME_LAG_2: item_ + 20],
                    title=str(str(final_X_index[index_])), 
                    zoom=False, 
                    range_=[TIME_LAG_2 - TIME_LAG_1_START, TIME_LAG_2 - TIME_LAG_1_END],
                    dir_=dir_
                   )

for index_, item_ in enumerate(final_X_index):
    diff = (final_test_data.iloc[index_ + TIME_LAG_2 + TIME_LAG_1_START - TIME_LAG_1_END + 1][['热风速度设定值', '筒壁1区温度设定值', '筒壁2区温度设定值']] - final_test_data.iloc[index_ + TIME_LAG_2 + TIME_LAG_1_START - TIME_LAG_1_END][['热风速度设定值', '筒壁1区温度设定值', '筒壁2区温度设定值']]).values
    print('{}, {} \t {}'.format(item_, np.round(pred[index_], 4), np.round(diff.astype(np.double), 4)) )
    
    
    

125, [-0.      0.0053 -0.0042] 	 [ 0.      0.0011 -0.0039]
126, [ 0.0001  0.0053 -0.0042] 	 [0.     0.0115 0.0115]
127, [ 0.0002  0.0053 -0.0042] 	 [0.     0.0173 0.012 ]
128, [ 0.0004  0.0053 -0.0042] 	 [0.     0.0448 0.0396]
129, [ 0.0006  0.0054 -0.0042] 	 [0.     0.0658 0.0623]
130, [ 0.0009  0.0054 -0.0042] 	 [0.     0.0534 0.0515]
131, [ 0.0009  0.0054 -0.0042] 	 [0.     0.0182 0.0183]
132, [ 0.0011  0.0054 -0.0042] 	 [ 0.     -0.0006 -0.0006]
133, [ 0.0014  0.0055 -0.0042] 	 [ 0.     -0.0108 -0.0038]
134, [ 0.0018  0.0055 -0.0042] 	 [ 0.     -0.0035 -0.0035]
135, [ 0.0023  0.0056 -0.0042] 	 [ 0.     -0.0241 -0.017 ]
136, [ 0.0024  0.0056 -0.0042] 	 [ 0.     -0.0445 -0.0351]
137, [ 0.0015  0.0055 -0.0042] 	 [ 0.     -0.0401 -0.0288]
138, [ 0.001   0.0054 -0.0042] 	 [ 0.     -0.0341 -0.0342]
139, [ 0.0011  0.0054 -0.0042] 	 [ 0.     -0.0756 -0.0624]
140, [ 0.0018  0.0055 -0.0042] 	 [ 0.     -0.051  -0.0391]
141, [ 0.0018  0.0055 -0.0042] 	 [ 0.     -0.0511 -0.0403]
142, [ 0.0017  

328, [ 0.0013  0.0055 -0.0042] 	 [0.     0.0388 0.0388]
329, [ 0.0012  0.0054 -0.0042] 	 [0.     0.0399 0.0433]
330, [ 0.0012  0.0054 -0.0042] 	 [0.0004 0.0282 0.0321]
331, [ 0.0011  0.0054 -0.0042] 	 [0.     0.0105 0.0112]
332, [ 0.0011  0.0054 -0.0042] 	 [0.0004 0.0244 0.0245]
333, [ 0.0011  0.0054 -0.0042] 	 [0.     0.0137 0.0157]
334, [ 0.001   0.0054 -0.0042] 	 [0.     0.0358 0.0358]
335, [ 0.0012  0.0054 -0.0042] 	 [0.0004 0.0105 0.0169]
336, [ 0.0013  0.0054 -0.0042] 	 [ 0.     -0.0121 -0.0038]
337, [ 0.0013  0.0055 -0.0042] 	 [0.     0.0229 0.0311]
338, [ 0.0012  0.0054 -0.0042] 	 [ 0.0004 -0.0019  0.005 ]
339, [ 0.0014  0.0055 -0.0042] 	 [ 0.     -0.0155 -0.0104]
340, [ 0.0017  0.0055 -0.0042] 	 [ 0.0004 -0.0234 -0.0234]
341, [ 0.002   0.0055 -0.0042] 	 [ 0.     -0.0303 -0.0267]
342, [ 0.0023  0.0056 -0.0042] 	 [ 0.     -0.0195 -0.0158]
343, [ 0.0022  0.0056 -0.0042] 	 [ 0.0004 -0.0373 -0.0351]
344, [ 0.0019  0.0055 -0.0042] 	 [ 0.     -0.0279 -0.0306]
345, [ 0.0016  0.0055 -0

523, [ 0.0012  0.0054 -0.0042] 	 [ 0.     -0.0408 -0.0434]
524, [ 0.001   0.0054 -0.0042] 	 [ 0.     -0.0337 -0.032 ]
525, [ 0.0008  0.0054 -0.0042] 	 [ 0.     -0.0279 -0.0264]
526, [ 0.0006  0.0054 -0.0042] 	 [ 0.     -0.0184 -0.0174]
527, [ 0.0004  0.0053 -0.0042] 	 [ 0.     -0.0116 -0.0087]
528, [ 0.0002  0.0053 -0.0042] 	 [0.     0.0141 0.0166]
529, [ 0.      0.0053 -0.0042] 	 [ 0.     -0.0001  0.0015]
530, [-0.      0.0053 -0.0042] 	 [ 0.     -0.011  -0.0065]
531, [ 0.0001  0.0053 -0.0042] 	 [ 0.     -0.0206 -0.0146]
532, [-0.      0.0053 -0.0042] 	 [ 0.     -0.0234 -0.0174]
533, [ 0.0004  0.0053 -0.0042] 	 [ 0.     -0.0056 -0.0008]
534, [ 0.0011  0.0054 -0.0042] 	 [0.     0.0276 0.0285]
535, [ 0.0008  0.0054 -0.0042] 	 [0.     0.0164 0.0217]
536, [ 0.0006  0.0054 -0.0042] 	 [0.     0.0058 0.0126]
537, [ 0.0006  0.0054 -0.0042] 	 [ 0.     -0.0214 -0.0153]
538, [ 0.0004  0.0053 -0.0042] 	 [ 0.     -0.0369 -0.0314]
539, [ 0.0007  0.0054 -0.0042] 	 [ 0.     -0.0299 -0.0254]
540, [ 0.

736, [ 0.0026  0.0056 -0.0042] 	 [0.     0.0415 0.0343]
737, [ 0.0023  0.0056 -0.0042] 	 [0.0002 0.04   0.0236]
738, [ 0.0023  0.0056 -0.0042] 	 [0.     0.0298 0.0279]
739, [ 0.0023  0.0056 -0.0042] 	 [0.0002 0.0362 0.0312]
740, [ 0.0023  0.0056 -0.0042] 	 [0.     0.0491 0.0454]
741, [ 0.0023  0.0056 -0.0042] 	 [0.     0.0514 0.0233]
742, [ 0.0022  0.0056 -0.0042] 	 [0.0002 0.0169 0.0047]
743, [ 0.0022  0.0055 -0.0042] 	 [ 0.      0.0043 -0.0182]
744, [ 0.002   0.0055 -0.0042] 	 [ 0.     -0.0113 -0.0402]
745, [ 0.0019  0.0055 -0.0042] 	 [ 0.0003 -0.046  -0.0574]
746, [ 0.0019  0.0055 -0.0042] 	 [ 0.     -0.0134 -0.0136]
747, [ 0.0019  0.0055 -0.0042] 	 [ 0.0006 -0.0071 -0.0379]
748, [ 0.0019  0.0055 -0.0042] 	 [ 0.     -0.0334 -0.0488]
749, [ 0.002   0.0055 -0.0042] 	 [ 0.     -0.0412 -0.056 ]
750, [ 0.002   0.0055 -0.0042] 	 [ 0.     -0.0259 -0.0387]
751, [ 0.0022  0.0056 -0.0042] 	 [ 0.0006 -0.0569 -0.0574]
752, [ 0.0021  0.0055 -0.0042] 	 [ 0.     -0.0498 -0.065 ]
753, [ 0.0021  0.0

905, [ 0.0016  0.0055 -0.0042] 	 [ 0.     -0.0728 -0.0578]
906, [ 0.0016  0.0055 -0.0042] 	 [ 0.     -0.0771 -0.0614]
907, [ 0.0016  0.0055 -0.0042] 	 [ 0.     -0.1069 -0.0921]
908, [ 0.0016  0.0055 -0.0042] 	 [ 0.     -0.0278 -0.0278]
909, [ 0.0017  0.0055 -0.0042] 	 [ 0.     -0.0559 -0.0441]
910, [ 0.0018  0.0055 -0.0042] 	 [ 0.     -0.0415 -0.0328]
911, [ 0.0018  0.0055 -0.0042] 	 [ 0.     -0.0297 -0.023 ]
912, [ 0.0017  0.0055 -0.0042] 	 [ 0.     -0.0154 -0.0097]
913, [ 0.0015  0.0055 -0.0042] 	 [0.     0.0142 0.0143]
914, [ 0.0014  0.0055 -0.0042] 	 [0.     0.     0.0036]
915, [ 0.0013  0.0054 -0.0042] 	 [0.     0.001  0.0034]
916, [ 0.0012  0.0054 -0.0042] 	 [ 0.     -0.0171 -0.0152]
917, [ 0.0011  0.0054 -0.0042] 	 [ 0.     -0.0007  0.0029]
918, [ 0.0009  0.0054 -0.0042] 	 [0.     0.0076 0.0125]
919, [ 0.0007  0.0054 -0.0042] 	 [ 0.     -0.0146 -0.0146]
920, [ 0.0007  0.0054 -0.0042] 	 [ 0.     -0.0053 -0.0017]
921, [ 0.0008  0.0054 -0.0042] 	 [ 0.     -0.0141 -0.0096]
922, [ 0.

1071, [ 0.0033  0.0057 -0.0042] 	 [ 0.     -0.0017 -0.0094]
1072, [ 0.0032  0.0057 -0.0042] 	 [ 0.     -0.007  -0.0198]
1073, [ 0.0031  0.0057 -0.0042] 	 [ 0.     -0.0189 -0.0135]
1074, [ 0.0029  0.0056 -0.0042] 	 [0.     0.0107 0.0276]
1075, [ 0.0027  0.0056 -0.0042] 	 [0.     0.0275 0.0202]
1076, [ 0.0026  0.0056 -0.0042] 	 [0.     0.0113 0.0136]
1077, [ 0.0024  0.0056 -0.0042] 	 [0.     0.0146 0.0156]
1078, [ 0.0021  0.0055 -0.0042] 	 [0.     0.0109 0.0111]
1079, [ 0.002   0.0055 -0.0042] 	 [0.     0.0139 0.0191]
1080, [ 0.0019  0.0055 -0.0042] 	 [ 0.      0.0053 -0.0139]
1081, [ 0.0018  0.0055 -0.0042] 	 [ 0.     -0.0155  0.0029]
1082, [ 0.0017  0.0055 -0.0042] 	 [0.     0.0296 0.0509]
1083, [ 0.0017  0.0055 -0.0042] 	 [0.     0.0473 0.046 ]
1084, [ 0.0017  0.0055 -0.0042] 	 [0.     0.0335 0.034 ]
1085, [ 0.0017  0.0055 -0.0042] 	 [0.     0.0399 0.0436]
1086, [ 0.0017  0.0055 -0.0042] 	 [0.     0.0302 0.0071]
1087, [ 0.0016  0.0055 -0.0042] 	 [ 0.     -0.0126 -0.0322]
1088, [ 0.001

1217, [ 0.0029  0.0056 -0.0042] 	 [ 0.     -0.0062 -0.0155]
1218, [ 0.0035  0.0057 -0.0042] 	 [0.0002 0.0302 0.0211]
1219, [ 0.0039  0.0058 -0.0042] 	 [0.     0.0302 0.022 ]
1220, [ 0.0034  0.0057 -0.0042] 	 [0.     0.0323 0.027 ]
1221, [ 0.0021  0.0055 -0.0042] 	 [0.0002 0.0206 0.0172]
1222, [ 0.0028  0.0056 -0.0042] 	 [0.     0.0128 0.0046]
1223, [ 0.003   0.0056 -0.0042] 	 [ 0.      0.0015 -0.0059]
1224, [ 0.0027  0.0056 -0.0042] 	 [0.0002 0.0308 0.0139]
1225, [ 0.0023  0.0056 -0.0042] 	 [0. 0. 0.]
1226, [ 0.0018  0.0055 -0.0042] 	 [0.0002 0.0263 0.0268]
1227, [ 0.0017  0.0055 -0.0042] 	 [0.     0.0254 0.017 ]
1228, [ 0.0017  0.0055 -0.0042] 	 [ 0.     -0.0063 -0.0131]
1229, [ 0.0016  0.0055 -0.0042] 	 [ 0.0002 -0.0129 -0.0183]
1230, [ 0.0016  0.0055 -0.0042] 	 [ 0.     -0.0237 -0.0283]
1231, [ 0.0017  0.0055 -0.0042] 	 [ 0.     -0.0171 -0.0239]
1232, [ 0.0016  0.0055 -0.0042] 	 [ 0.0002 -0.0206 -0.022 ]
1233, [ 0.0014  0.0055 -0.0042] 	 [ 0.     -0.0369 -0.0457]
1234, [ 0.0015  0.0

1359, [ 0.0008  0.0054 -0.0042] 	 [ 0.0002 -0.0651 -0.0531]
1360, [ 0.0011  0.0054 -0.0042] 	 [ 0.     -0.0514 -0.0398]
1361, [ 0.0013  0.0054 -0.0042] 	 [ 0.     -0.0229 -0.0096]
1362, [ 0.0013  0.0054 -0.0042] 	 [ 0.0002 -0.0309 -0.0309]
1363, [ 0.0013  0.0054 -0.0042] 	 [ 0.     -0.0278 -0.0146]
1364, [ 0.0013  0.0054 -0.0042] 	 [ 0.     -0.0197 -0.0089]
1365, [ 0.0014  0.0055 -0.0042] 	 [ 0.0002 -0.0274 -0.0185]
1366, [ 0.0014  0.0055 -0.0042] 	 [ 0.     -0.0395 -0.0317]
1367, [ 0.0016  0.0055 -0.0042] 	 [ 0.0002 -0.0252 -0.0205]
1368, [ 0.0017  0.0055 -0.0042] 	 [0.     0.0011 0.0012]
1369, [ 0.0019  0.0055 -0.0042] 	 [ 0.     -0.0078 -0.0094]
1370, [ 0.0021  0.0055 -0.0042] 	 [ 0.0002 -0.0125 -0.0174]
1371, [ 0.0021  0.0055 -0.0042] 	 [ 0.     -0.0222 -0.0271]
1372, [ 0.002   0.0055 -0.0042] 	 [ 0.     -0.0157 -0.0213]
1373, [ 0.0018  0.0055 -0.0042] 	 [ 0.0002  0.0053 -0.0031]
1374, [ 0.0017  0.0055 -0.0042] 	 [ 0.     -0.0124 -0.0124]
1375, [ 0.0018  0.0055 -0.0042] 	 [ 0.0002 

In [260]:
def generate_real_test_data(item):
    length = len(item)
    final_X_test_ = []
    final_X_index_ = []
    
    for item_index in range(TIME_LAG_2 + TIME_LAG_1_START, length - STABLE_WINDOWS_SIZE, 1):
        final_X_test_.append(calc_feature(item, item_index))
        final_X_index_.append(item_index)
        
    return np.array(final_X_test_), np.array(final_X_index_)

final_test_data = split_data_per_number[1][5]
final_X_test, final_X_index = generate_real_test_data(final_test_data)
pred = model.predict(final_X_test)

dir_ = 'final/' + str(datetime.now())

# Plot
for index_, item_ in enumerate(final_X_index):
    draw_split_data(final_test_data[item_- TIME_LAG_2: item_ + 20],
                    title=str(str(final_X_index[index_])), 
                    zoom=False, 
                    range_=[TIME_LAG_2 - TIME_LAG_1_START, TIME_LAG_2 - TIME_LAG_1_END],
                    dir_=dir_
                   )

for index_, item_ in enumerate(final_X_index):
    diff = (final_test_data.iloc[index_ + TIME_LAG_2 + TIME_LAG_1_START - TIME_LAG_1_END + 1][['热风速度设定值', '筒壁1区温度设定值', '筒壁2区温度设定值']] - final_test_data.iloc[index_ + TIME_LAG_2 + TIME_LAG_1_START - TIME_LAG_1_END][['热风速度设定值', '筒壁1区温度设定值', '筒壁2区温度设定值']]).values
    print('{}, {} \t {}'.format(item_, np.round(pred[index_], 4), np.round(diff.astype(np.double), 4)) )
    
    
    

125, [-0.     -0.0431 -0.044 ] 	 [0.     0.0108 0.0065]
126, [-0.     -0.0454 -0.0478] 	 [0.     0.01   0.0127]
127, [-0.     -0.0487 -0.0521] 	 [0.     0.0119 0.0119]
128, [-0.     -0.0511 -0.0548] 	 [ 0.     -0.008  -0.0002]
129, [-0.     -0.0519 -0.0555] 	 [0.     0.0027 0.0073]
130, [-0.     -0.0519 -0.0554] 	 [0.     0.0335 0.0674]
131, [-0.     -0.0518 -0.0553] 	 [0.     0.0396 0.0244]
132, [-0.     -0.0506 -0.0537] 	 [0.     0.0611 0.0672]
133, [-0.     -0.0453 -0.0487] 	 [0.     0.0612 0.0612]
134, [-0.     -0.0461 -0.0494] 	 [0.     0.0453 0.048 ]
135, [-0.     -0.0478 -0.0509] 	 [0.     0.036  0.0346]
136, [-0.     -0.0504 -0.0534] 	 [0.     0.0466 0.0424]
137, [-0.     -0.0522 -0.0552] 	 [0.     0.0453 0.0675]
138, [-0.     -0.0543 -0.057 ] 	 [0.     0.0712 0.0289]
139, [ 0.     -0.0557 -0.0578] 	 [0.     0.0567 0.0567]
140, [ 0.     -0.0604 -0.0624] 	 [0.     0.0491 0.0346]
141, [-0.     -0.0658 -0.0681] 	 [ 0.     -0.0111 -0.057 ]
142, [-0.     -0.068  -0.0704] 	 [0.     0

320, [0.     0.0081 0.0066] 	 [ 0.     -0.0278 -0.0267]
321, [0.     0.0109 0.009 ] 	 [ 0.     -0.0395 -0.0368]
322, [0.     0.0148 0.0122] 	 [ 0.     -0.0411 -0.0349]
323, [0.     0.0184 0.0143] 	 [ 0.     -0.0179 -0.0101]
324, [0.     0.0218 0.0152] 	 [ 0.     -0.0329 -0.0249]
325, [0.     0.0253 0.0157] 	 [ 0.     -0.0012 -0.0012]
326, [0.     0.029  0.0172] 	 [ 0.     -0.0228 -0.0127]
327, [0.     0.0333 0.0205] 	 [ 0.     -0.0259 -0.0164]
328, [0.     0.036  0.0261] 	 [ 0.     -0.0375 -0.0298]
329, [0.     0.0395 0.0276] 	 [ 0.     -0.0469 -0.0396]
330, [0.     0.0438 0.0291] 	 [ 0.     -0.0439 -0.0377]
331, [0.     0.0458 0.0322] 	 [ 0.     -0.0215 -0.0215]
332, [0.     0.0469 0.0457] 	 [ 0.     -0.0287 -0.0271]
333, [0.     0.0505 0.0523] 	 [ 0.     -0.0133 -0.0163]
334, [0.     0.0547 0.0574] 	 [ 0.     -0.0269 -0.0311]
335, [0.     0.0556 0.0579] 	 [ 0.     -0.0379 -0.0431]
336, [0.     0.0521 0.0537] 	 [ 0.     -0.0149 -0.0201]
337, [0.     0.055  0.0561] 	 [ 0.     -0.0245 -

536, [0.     0.0434 0.0469] 	 [ 0.     -0.0947 -0.0932]
537, [0.     0.0507 0.0546] 	 [ 0.     -0.0519 -0.0536]
538, [0.     0.0591 0.0621] 	 [ 0.     -0.0608 -0.0606]
539, [0.     0.0687 0.0695] 	 [ 0.     -0.0274 -0.0317]
540, [0.     0.079  0.0777] 	 [ 0.     -0.0146 -0.0191]
541, [0.     0.088  0.0854] 	 [ 0.     -0.0243 -0.027 ]
542, [0.     0.0965 0.0934] 	 [ 0.     -0.0252 -0.0246]
543, [0.     0.1046 0.1014] 	 [ 0.     -0.0405 -0.0359]
544, [0.     0.1116 0.1085] 	 [ 0.     -0.0075 -0.0069]
545, [0.     0.1171 0.1141] 	 [ 0.     -0.0224 -0.0179]
546, [0.     0.1215 0.1185] 	 [ 0.     -0.0255 -0.0223]
547, [0.     0.1266 0.1239] 	 [ 0.     -0.0081 -0.0068]
548, [0.     0.1308 0.128 ] 	 [0.     0.0001 0.0034]
549, [0.     0.134  0.1311] 	 [ 0.     -0.0238 -0.0171]
550, [0.     0.1375 0.1346] 	 [ 0.     -0.0328 -0.0322]
551, [0.     0.1416 0.1385] 	 [ 0.     -0.0596 -0.0525]
552, [0.     0.145  0.1413] 	 [ 0.     -0.0456 -0.0375]
553, [0.     0.1384 0.136 ] 	 [ 0.     -0.0228 -0.0

707, [ 0.      0.0005 -0.0028] 	 [ 0.     -0.0168 -0.0273]
708, [ 0.     -0.0012 -0.0047] 	 [ 0.     -0.0384 -0.0448]
709, [ 0.     -0.0013 -0.0051] 	 [ 0.     -0.0337 -0.037 ]
710, [ 0.     -0.0004 -0.0044] 	 [ 0.     -0.0338 -0.0371]
711, [ 0.      0.     -0.0043] 	 [ 0.     -0.0091 -0.0129]
712, [ 0.      0.0001 -0.0045] 	 [0.     0.0049 0.0179]
713, [ 0.     -0.001  -0.0058] 	 [0.     0.023  0.0082]
714, [ 0.     -0.0019 -0.0071] 	 [ 0.     -0.0039 -0.0037]
715, [ 0.     -0.0014 -0.0067] 	 [0.     0.0111 0.0128]
716, [ 0.     -0.0009 -0.0055] 	 [0.     0.0074 0.0104]
717, [ 0.     -0.0004 -0.004 ] 	 [0.     0.0076 0.0143]
718, [ 0.     -0.0006 -0.0026] 	 [ 0.     -0.0109 -0.0025]
719, [ 0.     -0.0008 -0.002 ] 	 [ 0.     -0.0234 -0.0233]
720, [ 0.     -0.0005 -0.0012] 	 [ 0.     -0.0403 -0.0316]
721, [0.     0.0006 0.0005] 	 [ 0.     -0.0346 -0.0237]
722, [0.     0.0039 0.004 ] 	 [ 0.     -0.0356 -0.024 ]
723, [0.     0.0078 0.008 ] 	 [ 0.     -0.0291 -0.0189]
724, [0.     0.0108 0

916, [-0.     -0.0256 -0.0272] 	 [ 0.     -0.013  -0.0193]
917, [-0.     -0.0233 -0.0223] 	 [ 0.     -0.0047 -0.0073]
918, [-0.     -0.0201 -0.0182] 	 [ 0.     -0.0049 -0.0049]
919, [-0.     -0.0162 -0.0148] 	 [ 0.     -0.0304 -0.0279]
920, [-0.     -0.0114 -0.0115] 	 [ 0.     -0.0073 -0.0018]
921, [-0.     -0.0068 -0.0084] 	 [ 0.     -0.0329 -0.0275]
922, [ 0.     -0.0028 -0.0052] 	 [ 0.     -0.004   0.0001]
923, [-0.      0.0006 -0.0021] 	 [0.     0.0095 0.0154]
924, [-0.     -0.0003 -0.0029] 	 [ 0.     -0.0164 -0.0164]
925, [-0.     -0.0024 -0.0046] 	 [ 0.     -0.0436 -0.0369]
926, [-0.     -0.0017 -0.0038] 	 [ 0.     -0.0401 -0.035 ]
927, [-0.      0.0004 -0.0017] 	 [ 0.     -0.0329 -0.0291]
928, [-0.     0.003  0.001] 	 [ 0.     -0.007  -0.0052]
929, [-0.      0.0052  0.0034] 	 [0.     0.0119 0.0171]
930, [-0.      0.0056  0.0039] 	 [0.     0.034  0.0342]
931, [-0.      0.0068  0.0051] 	 [0.     0.053  0.0521]
932, [-0.      0.0091  0.0075] 	 [0.     0.0102 0.0097]
933, [-0.      